<div class="alert alert-danger">
**Due date:** 2017-01-27
</div>

# Lab 1: Text Classification

**Students:** Johan Lindström (johli160), Jonathan Sjölund (jonsj507)

In this lab you will implement and compare the performance of two simple text classifiers: a Naive Bayes classifier and a classifier based on the averaged perceptron.

The data set that you will use in this lab is the [review polarity data set](https://www.cs.cornell.edu/people/pabo/movie-review-data/) first used by [Pang and Lee (2004)](http://www.aclweb.org/anthology/P04-1035). This data set consists of 2,000 movie reviews, each of which has been tagged as either positive or negative towards the movie at hand. The data is originally distributed as a collection of text files. For this lab we have put all files into two JSON files, one for training and one for testing.

## Introduction

Start by importing the module for this lab.

In [1]:
import nlp1

The next cell loads the training data and the test data:

In [2]:
training_data = nlp1.load_data("/home/TDDE09/labs/nlp1/review_polarity.train.json")
test_data = nlp1.load_data("/home/TDDE09/labs/nlp1/review_polarity.test.json")

As you will see, each data instance is a pair whose first component is a document, represented as a list of tokens, and whose second component is the gold-standard polarity of the review&nbsp;&ndash; either positive (`pos`) or negative (`neg`).

In [3]:
print(training_data[813])

(['this', 'film', 'is', 'extraordinarily', 'horrendous', 'and', "i'm", 'not', 'going', 'to', 'waste', 'any', 'more', 'words', 'on', 'it', '.'], 'neg')


The two classifiers that you will implement in this lab should inherit from the following class, whose only method `predict` takes a document and returns the predicted class for that document (here: the polarity).

In [4]:
class Classifier(object):

    def predict(self, d):
        return None

## Evaluation

The first thing that you will have do is to implement a function

`accuracy(classifier, data)`

that computes the accuracy of a classifier on test data.

In [5]:
def accuracy(classifier, data):
    # TODO: Replace the following line with your own code to solve Problem 1
    correct = 0
    for x in range(0, len(data)):
        if(classifier.predict(data[x][0]) == data[x][1]):
            correct += 1
    return(correct/len(data))
    #return nlp1.accuracy(classifier, data)

You can test this function by computing the accuracy of a Naive Bayes classifier on the test data:

In [6]:
classifier = nlp1.NaiveBayesClassifier.train(training_data)
print(accuracy(classifier, test_data))

0.765


<div class="panel panel-primary">
<div class="panel-heading">Problem 1</div>
<div class="panel-body">
Provide your own implementation of the `accuracy()` function. Test your implementation by redoing the evaluation. You should get exactly the same results as before.
</div>
</div>

**Hint:** Using an appropriate function from the `statistics` module, this problem can be solved in a one-liner.

## Naive Bayes classifier

To implement the Naive Bayes classifier, you should complete the following code:

In [7]:
class MyNaiveBayesClassifier(Classifier):

    def predict(self, d):
        # TODO: Replace the following line with your own code to solve Problem 2
        import math
        negScore = math.log(probNeg,10)
        posScore = math.log(probPos,10)
        
        for x in range(0,len(d)):
            
            if(d[x] in dictNeg):
                negScore = negScore+math.log(dictNeg[d[x]]/fwcNeg,10)
            else:
                negScore = negScore+math.log(1/fwcNeg,10)
                   
            if(d[x] in dictPos):
                posScore = posScore+math.log(dictPos[d[x]]/fwcPos,10)
            else:
                posScore = posScore+math.log(1/fwcPos,10)

            
        if(negScore > posScore):
            return 'neg'
        else:
            return 'pos'

    
    @classmethod
    def train(cls, data, k=1):
        # TODO: Replace the following line with your own code to solve Problem 2
        numOfNeg = 0
        listOfNegWord = []
        listOfPosWord = []
        
        global dictNeg, dictPos, probNeg, probPos,fwcNeg,fwcPos
        dictNeg = {}
        dictPos = {}
        
        for x in range(0,len(data)):
            currList = data[x][0]
            if(data[x][1] == 'neg'):
                numOfNeg += 1
                listOfNegWord = listOfNegWord + currList
                for y in range(0,len(currList)):
                    if(currList[y] in dictNeg):
                        dictNeg.update({currList[y]:dictNeg[currList[y]]+1})
                    else:
                        dictNeg.update({currList[y]:1+k})
                                       
            else:
                listOfPosWord = listOfPosWord + currList
                for y in range(0,len(currList)):
                    if(currList[y] in dictPos):
                        dictPos.update({currList[y]:dictPos[currList[y]]+1})
                    else:
                        dictPos.update({currList[y]:1+k})
             
        probNeg = numOfNeg/len(data)   #P(c)
        probPos = 1-probNeg  
        
        listOfAllWords = listOfNegWord+listOfPosWord
        uniqueAllWords = set(listOfAllWords)
        uniqueAllWords = list(uniqueAllWords)
        numOfUniqAllWords = len(uniqueAllWords)
        
        numOfNegWord = len(listOfNegWord)
        fwcNeg =  numOfNegWord+k*numOfUniqAllWords  # frequency of all words amongst all negative documents
        
        numOfPosWord = len(listOfPosWord)
        fwcPos =  numOfPosWord+k*numOfUniqAllWords  # frequency of all words amongst all negative documents
    
        return cls()

In this skeleton the method `predict()` should implement the Naive Bayes classification rule. The method `train()` should return a new classifier that has been trained on the specified training data using maximum likelihood estimation with add-$k$ smoothing.

To test your implementation, you can re-do the evaluation from above:

In [8]:
classifier1 = MyNaiveBayesClassifier.train(training_data)
print(accuracy(classifier1, test_data))

0.765


<div class="panel panel-primary">
<div class="panel-heading">Problem 2</div>
<div class="panel-body">
Implement the two methods in `MyNaiveBayesClassifier`. Test your implementation by evaluating on the test data. Your results should be very similar to the ones that you got when you evaluated your accuracy function in Problem&nbsp;1.
</div>
</div>

## Averaged perceptron classifier

Here is the code skeleton for the averaged perceptron classifier:

In [9]:
class MyPerceptronClassifier(Classifier):

    def predict(self, x):
        # TODO: Replace the following line with your own code to solve Problem 3
        xDict = {}
        for i in range(0,len(x)):
            if(x[i] in xDict):
                xDict.update({x[i]:xDict[x[i]]+1})
            else:
                xDict.update({x[i]:1})
        
        pNeg = 0
        pPos = 0
                
        for word in xDict:
            if(word in self.weights['neg']):
                 pNeg = pNeg + self.weights['neg'][word]
            if(word in self.weights['pos']):   
                 pPos = pPos + self.weights['pos'][word]
                
            if(pNeg >= pPos):
                 p = 'neg'
            else:
                p = 'pos'
         
        return p
        

    @classmethod
    def train(cls, data, n_epochs=1):
        # TODO: Replace the following line with your own code to solve Problem 3
        perc = cls()
        dictNeg = {}
        dictPos = {}
        perc.classes = ['pos', 'neg']
        
        ######## Only used to get every word that are in the negative and positive review #########
        for x in range(0,len(data)):
            currList = data[x][0]
            if(data[x][1] == 'neg'):
                for y in range(0,len(currList)):
                    if(currList[y] in dictNeg):
                        dictNeg.update({currList[y]:dictNeg[currList[y]]+1})
                    else:
                        dictNeg.update({currList[y]:1})
                                       
            else:
                for y in range(0,len(currList)):
                    if(currList[y] in dictPos):
                        dictPos.update({currList[y]:dictPos[currList[y]]+1})
                    else:
                        dictPos.update({currList[y]:1})
        
        ###########################################################################################
        
        tempPos = dictPos
        tempNeg = dictNeg
        for k in dictPos:
            tempPos[k] = 0
        for k in dictNeg:
            tempNeg[k] = 0
            
        perc.weights = {'pos' : {}, 'neg' : {}} #Two different dictionaries for each class
        perc.weights.update({'pos':tempPos})
        perc.weights.update({'neg':tempNeg})
        
        acc = {'pos':{}, 'neg':{}} #dictionary of dictionaries
        acc.update({'pos':tempPos})
        acc.update({'neg':tempNeg})
        
        count = 1
        for e in range(n_epochs):
            for x,y in data:
                #x is a document, should be transformed from a list to a dictionary
                #y is a binary value for the gold standard class for the document
                xDict = {}
                for i in range(0,len(x)):
                    if(x[i] in xDict):
                        xDict.update({x[i]:xDict[x[i]]+1})
                    else:
                        xDict.update({x[i]:1})
        
                pNeg = 0
                pPos = 0
                
                p = perc.predict(x) # Predict the class
                
                if(p != y):
                    for word in xDict:  #for-loops with dictionary
                        if(word in perc.weights[p]):
                            perc.weights[p][word] -= 1
                            acc[p][word] -= count
                        if(word in perc.weights[y]):       
                            perc.weights[y][word] += 1
                            acc[y][word] += count
                count += 1 
        for k in perc.classes:
            for word in xDict:  #for-loops with dictionary
                if(word in perc.weights[k]):
                    z=1 # ignore this
                    perc.weights[k][word] = perc.weights[k][word] - acc[k][word]/count #averaging
        return perc

In this skeleton, the method `predict()` should implement the perceptron classification rule. The method `train()` should return a new classifier that has been trained on the specified training data using averaged perceptron training for the specified number of epochs.

To test your implementation, as before you can train a classifier on the training data and evaluate it on the test data:

In [10]:
classifier2 = MyPerceptronClassifier.train(training_data)
print(accuracy(classifier2, test_data))

0.785


<div class="panel panel-primary">
<div class="panel-heading">Problem 3</div>
<div class="panel-body">
Implement the two methods in `MyPerceptronClassifier`. Test your implementation by evaluating on the test data. You should get results in the 70&ndash;80% range. What happens if you repeat the experiment but do not do averaging? What happens when you train the classifier for two epochs? Enter your results into the table below.
</div>
</div>

<table>
<tr><td></td><td>averaging</td><td>no averaging</td></tr>
<tr><td>1 epoch</td><td>0.785</td><td>0.785</td></tr>
<tr><td>2 epochs</td><td>0.82</td><td>0.82</td></tr>
</table>

The reason we may get the same result for averaging and no averaging may be because the problem at hand is not linearly separable and since averaging is limited to only those problems the averaging here may or may not help at all. Which is the case here where there is no difference between using and not using averaging.

## Switching to binary features

In the lab so far, a document is represented as a list of the words that occur in it. For sentiment classification, several authors have suggested that a *binary* document representation, where each word is represented only once, can produce better results. In the last problem you will try to confirm this finding.

Your task is to implement a function `binarize()` that converts data into the binary representation:

In [11]:
def binarize(data):
    # TODO: Replace the following line to solve Problem 4
    temp = {}

    for i in range(0,len(data)):
        temp = data[i][0]
        temp = set(temp)
        temp = list(temp)
        data[i] = (temp,data[i][1])
        
    return data

The function is to be used in the following context:

In [12]:
binarized_training_data = binarize(training_data)
binarized_test_data = binarize(test_data)

classifier3 = MyNaiveBayesClassifier.train(binarized_training_data)
print(accuracy(classifier3, binarized_test_data))

classifier4 = MyPerceptronClassifier.train(binarized_training_data)
print(accuracy(classifier4, binarized_test_data))

0.805
0.785


<div class="panel panel-primary">
<div class="panel-heading">Problem 4</div>
<div class="panel-body">
Implement the `binarize()` function and run the evaluation. What do you observe? Summarise your results in one or two sentences.
</div>
</div>

Naive Bayes is better because it is more reasonable to look at the frequency of words over documents instead of frequency per document since frequency of stop words increases alot for bigger documents and are not relevant at all when classifying, with binarization we minimize this thus giving a more fair score. 
For the averaging perceptron classifier the result does not change with or without averaging which is logical in a sense since we do not look at frequency of words because we use dictionary, only occurrences, which means that we already look at binary values.